In [11]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import sys
sys.path.insert(0, '/home/er647/projects/feature-wise-active-learning')
sys.path.insert(0, '/home/er647/projects/feature-wise-active-learning/src')
sys.path.insert(0, '/home/er647/projects/feature-wise-active-learning/notebooks')

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wandb
import datetime
from utils import *
from _utils import filter_df_on_conditions, make_subplots
import seaborn as sns
from functools import partial

In [ ]:
datetime.datetime.now(datetime.timezone.utc).isoformat()

In [14]:
api = wandb.Api(timeout=60)

In [15]:
#### Get runs by tag
runs = api.runs('evangeorgerex/fwal', {
	"tags": '264b29f6',
	'State': 'finished'
})

# #### Get one run
# run = api.run('andreimargeloiu/low-data/rwzd9fud')

# #### Get runs by date
# runs = api.runs('andreimargeloiu/low-data', {
# 	"$and": [{
# 		'created_at': {
# 			"$lt": '2022-03-09T10',
# 			"$gt": '2022-03-07T17'
# 		},
# 		'config.embedding_type': 'nmf',
# 	}],
# 	'$or': [
# 		{'tags': '!!!!'},
# 		{'tags': '!!!!'},
# 		{'tags': '!!!!'},
# 		{
# 			"tags": '8e85015a',  # nmf size 50 
# 			'config.gamma': {'$eq': 0}
# 		}
	
# 	],
# 	'State': 'finished',
# })

print(len(runs))
runs = filter_runs(runs, return_dataframe=False)
print(f"Found {len(runs)} runs")
runs_df = transforms_runs_into_dataframe(runs)
print(runs_df.shape)

217
Found 217 runs
(217, 154)


In [ ]:
### Optional commands
# add_tag_and_update(runs, TAG)

# - run.created_at
# - run.config.keys()
# - run.summary._json_dict['bestmodel_train/balanced_accuracy']

In [11]:
runs_df.columns

Index(['name', 'id', '_timestamp', 'bestmodel_test_y_hat',
       'bestmodel_test/balanced_accuracy', 'bestmodel_test/precision_weighted',
       'bestmodel_test/F1_weighted', 'bestmodel_train/cross_entropy_loss',
       '_runtime', 'train/total_loss',
       ...
       'patience_early_stopping', 'concrete_anneal_iterations',
       'cosine_warm_restart_t_mult', 'cosine_warm_restart_eta_min',
       'sparsity_gene_embedding_size', 'sparsity_gene_embedding_type',
       'nonlinearity_weight_predictor', 'evaluate_with_sampled_datasets',
       'sparsity_regularizer_hyperparam', 'layers_for_hidden_representation'],
      dtype='object', length=152)

In [7]:
pd.set_option('display.max_columns', None)
runs_df[
    ['id',
    #  'lr','gamma',
     'mask_type',
     'sparsity_regularizer_hyperparam',
    #  'bestmodel_valid/balanced_accuracy','bestmodel_valid/reconstruction_loss','bestmodel_valid/sparsity_loss',
     'best_mask_parameters']
    ]

,id,mask_type,sparsity_regularizer_hyperparam,best_mask_parameters
0,ikhki7dh,probabilistic,1000.0,"[0.3366183936595917, 0.1287771463394165, 0.234..."
1,aqe3jozq,probabilistic,1000.0,"[0.3365437388420105, 0.12873981893062592, 0.23..."
2,wiy9fpyv,probabilistic,1000.0,"[0.3365899324417114, 0.12876291573047638, 0.23..."
3,ek3m1i82,probabilistic,1000.0,"[0.3365899324417114, 0.12876291573047638, 0.23..."
4,begatpu9,probabilistic,1000.0,"[0.33658725023269653, 0.12876157462596893, 0.2..."
...,...,...,...,...
195,zz8x59c5,sigmoid,0.0,"[0.3403615653514862, 0.24742859601974487, 0.38..."
196,kv4burnd,sigmoid,0.0,"[0.3523191809654236, 0.2648531198501587, 0.382..."
197,99wqap9g,sigmoid,0.0,"[0.37576863169670105, 0.2500225603580475, 0.36..."
198,k5g0jq4i,sigmoid,0.0,"[0.3704529404640198, 0.2280469834804535, 0.353..."


In [16]:
# Splitting the array in 'best_mask_parameters' into individual columns
parameters_df = pd.DataFrame(runs_df['best_mask_parameters'].to_list(), columns=[f'parameter_{i}' for i in range(1, 6)])
parameters_df = pd.concat([runs_df, parameters_df], axis=1)



In [23]:
df = parameters_df
columns = [f'parameter_{i}' for i in range(1, 6)]
columns.extend(['mask_type','dataset',
                # 'repeat_id','test_split',
                'bestmodel_valid/balanced_accuracy','bestmodel_valid/reconstruction_loss','bestmodel_valid/sparsity_loss'])
df = df[df['sparsity_regularizer_hyperparam']==1]
df[columns].groupby(['mask_type','dataset']).agg('mean')

parameter_1  parameter_2  parameter_3  \
mask_type     dataset                                                    
probabilistic simple_synth          0.336579     0.128757     0.234387   
              very_simple_synth     0.336576     0.128756     0.234386   
sigmoid       simple_synth         -1.943436    -1.864626    -2.083116   
              very_simple_synth    -2.313782    -2.586958    -2.585783   

                                 parameter_4  parameter_5  \
mask_type     dataset                                       
probabilistic simple_synth          0.230258    -1.122440   
              very_simple_synth     0.230256    -1.122429   
sigmoid       simple_synth         -2.043080    -3.346113   
              very_simple_synth    -2.505562    -3.927479   

                                 bestmodel_valid/balanced_accuracy  \
mask_type     dataset                                                
probabilistic simple_synth                                0.914035   
              very_simple_synth                           0.930208   
sigmoid       simple_synth                                0.980435   
              very_simple_synth                           0.995750   

                                 bestmodel_valid/reconstruction_loss  \
mask_type     dataset                                                  
probabilistic simple_synth                                  0.183964   
              very_simple_synth                             0.172642   
sigmoid       simple_synth                                  0.009447   
              very_simple_synth                             0.001652   

                                 bestmodel_valid/sparsity_loss  
mask_type     dataset                                           
probabilistic simple_synth                            2.518277  
              very_simple_synth                       2.496519  
sigmoid       simple_synth                            0.943999  
              very_simple_synth                       0.413094

In [26]:
sigmoid = lambda x: 1 / (1 + np.exp(-x))
sigmoid_df = df
sigmoid_df[["parameter_1", "parameter_2", "parameter_3", "parameter_4", "parameter_5"]] = sigmoid_df[["parameter_1", "parameter_2", "parameter_3", "parameter_4", "parameter_5"]].apply(sigmoid)
sigmoid_df[columns].groupby(['mask_type','dataset']).agg('mean')

/tmp/ipykernel_4138728/3650782278.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sigmoid_df[["parameter_1", "parameter_2", "parameter_3", "parameter_4", "parameter_5"]] = sigmoid_df[["parameter_1", "parameter_2", "parameter_3", "parameter_4", "parameter_5"]].apply(sigmoid)
/tmp/ipykernel_4138728/3650782278.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sigmoid_df[["parameter_1", "parameter_2", "parameter_3", "parameter_4", "parameter_5"]] = sigmoid_df[["parameter_1", "parameter_2", "parameter_3", 

parameter_1  parameter_2  parameter_3  \
mask_type     dataset                                                    
probabilistic simple_synth          0.583359     0.532145     0.558330   
              very_simple_synth     0.583359     0.532145     0.558330   
sigmoid       simple_synth          0.126166     0.134857     0.111661   
              very_simple_synth     0.090106     0.070077     0.070227   

                                 parameter_4  parameter_5  \
mask_type     dataset                                       
probabilistic simple_synth          0.557311     0.245559   
              very_simple_synth     0.557311     0.245561   
sigmoid       simple_synth          0.115670     0.034357   
              very_simple_synth     0.075567     0.019369   

                                 bestmodel_valid/balanced_accuracy  \
mask_type     dataset                                                
probabilistic simple_synth                                0.914035   
              very_simple_synth                           0.930208   
sigmoid       simple_synth                                0.980435   
              very_simple_synth                           0.995750   

                                 bestmodel_valid/reconstruction_loss  \
mask_type     dataset                                                  
probabilistic simple_synth                                  0.183964   
              very_simple_synth                             0.172642   
sigmoid       simple_synth                                  0.009447   
              very_simple_synth                             0.001652   

                                 bestmodel_valid/sparsity_loss  
mask_type     dataset                                           
probabilistic simple_synth                            2.518277  
              very_simple_synth                       2.496519  
sigmoid       simple_synth                            0.943999  
              very_simple_synth                       0.413094

In [ ]:
### Plot
pd.options.display.float_format = '{:,.2f}'.format

grouped = runs_df.groupby(['dataset', 'embedding_type'])
g1 = grouped.agg({'bestmodel_train/balanced_accuracy': ['mean', 'std']})*100
g2 = grouped.agg({'bestmodel_test/balanced_accuracy': ['mean', 'std']})*100
res = pd.concat([g1, g2], axis=1).round(decimals=2)
res

In [ ]:
# file_name = 'test'
# res.to_csv(f'/home/er647/projects/feature-wise-active-learning/compute_results/csv/{file_name}.csv')
# res.to_excel(f'/home/er647/projects/feature-wise-active-learning/compute_results/csv/{file_name}.xlsx')